# Filter terms
- new in GAF T0
- with low frequency

### Imports, drive, directories

In [1]:
import numpy as np
import pandas as pd

In [2]:
!git clone https://github.com/sinc-lab/exp2GO.git

Cloning into 'exp2GO'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 155 (delta 52), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (155/155), 38.84 MiB | 17.40 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [3]:
path = 'exp2GO/data/'

### Load data

In [4]:
species = 'ara' # ara dicty yeast

if species=='ara':
  gafnr_1 = '131'
  gafnr00 = '138'
else:
  gafnr_1 = '59'
  gafnr00 = '66'

subont = 'BP'

In [5]:
annotationsT_1 = pd.read_csv(path + species + '_terms_gaf'+ gafnr_1 +
                           '_'+subont+'_with_expr_EXP.csv',
                           header=None, index_col=0)

annotationsT_1.fillna('',inplace=True)
annotationsT_1.index= [x.upper() for x in annotationsT_1.index]
labelsT_1 = np.unique(annotationsT_1.to_numpy().ravel())
labelsT_1=labelsT_1[1:]

In [6]:
annotationsT0 = pd.read_csv(path + species + '_terms_gaf'+ gafnr00 +
                          '_'+subont+'_with_expr_EXP.csv',
                          header=None, index_col=0)

annotationsT0 = annotationsT0.dropna(axis=0, how='all') # drop NA values 
annotationsT0.fillna('',inplace=True)
annotationsT0.index= [x.upper() for x in annotationsT0.index]
labelsT0 = np.unique(annotationsT0.to_numpy().ravel())
labelsT0=labelsT0[1:]

### Filter with T0 genes

In [7]:
genes = list(annotationsT0.index)
print(len(genes))

656


In [8]:
annotationsT_1 = annotationsT_1.loc[genes]

### Remove new terms in T0

In [9]:
alllabels = list(set(labelsT_1).intersection(set(labelsT0)))

print('Labels in T-1: ', len(labelsT_1))
print('Labels in T0:  ', len(labelsT0))
print('All labels:    ', len(alllabels))

Labels in T-1:  687
Labels in T0:   724
All labels:     681


### Remove terms with less than 3 counts 

In [10]:
term_names, term_counts = np.unique(annotationsT_1.values, return_counts=True)
remove_terms = term_names[term_counts<3]

alllabels = list(set(alllabels) - set(remove_terms))

print('Removed terms:  ', remove_terms.shape)
print('Remained terms: ', len(alllabels))

Removed terms:   (540,)
Remained terms:  145


### Process annotation lists for T-1 and T0

In [11]:
annotationsT_1_orig = annotationsT_1.copy()
annotationsT0_orig = annotationsT0.copy()

In [12]:
for gene in annotationsT_1.index:
  termsOK = list(set(annotationsT_1.loc[gene]).intersection(alllabels))
  if (len(set(annotationsT_1.loc[gene])) - len(termsOK)) > 1: # for '' in ancestors labels
    annotationsT_1.loc[gene][:len(termsOK)] = termsOK
    annotationsT_1.loc[gene][len(termsOK):] = ''

  termsOK = list(set(annotationsT0.loc[gene]).intersection(alllabels))
  if (len(set(annotationsT0.loc[gene])) - len(termsOK)) > 1:
    annotationsT0.loc[gene][:len(termsOK)] = termsOK
    annotationsT0.loc[gene][len(termsOK):] = ''

In [13]:
annotationsT_1.loc[annotationsT_1[1]!=''].shape

(497, 36)

In [14]:
annotationsT_1.to_csv(path + species + '_terms_gaf'+ gafnr_1 + 
                    '_'+subont+'_with_expr_EXP_FILTERED_terms.csv',
                    header=None)

In [15]:
annotationsT0.to_csv(path + species + '_terms_gaf'+ gafnr00 + 
                   '_'+subont+'_with_expr_EXP_FILTERED_terms.csv',
                   header=None)